# **Function Calling to Generate JSON data**

 #### **✅ Problem:**

LLMs often generate unstructured or loosely structured responses. This makes it hard to reliably extract data for downstream processing. Simply prompting the model to output JSON is not reliable, especially in complex scenarios.

**Challenge:** How can we get structured, validated JSON output reliably from a language model?


####**🛠️ Solution:**

This lab demonstrates the use of OpenAI Function Calling, a mechanism where:

- You define a function signature (including parameter names, types, and descriptions).

- The model is guided to generate arguments in JSON format that match the signature.

- OpenAI automatically parses this into structured data you can pass to real functions.


Think of this as a contract-driven way of interacting with LLMs.

##**0. Install Dependencies**

In [2]:
!pip install openai pydantic

##**2. Get API key from Secret**

In [3]:
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('OPENAI_API_KEY')

##**3. Python Code**

In [28]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Pune. she earns $35000 per month",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                        "salary": {"type": "integer"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age","salary","city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Extract the structured response
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments

import json
parsed = json.loads(arguments)
print(parsed)

{'name': 'Ritu Patil', 'age': 30, 'salary': 35000, 'city': 'Pune'}


In [30]:
print("Yearly Bonus = $", parsed['salary']*2)

Yearly Bonus = $ 70000


#**Validate using Pydantic**

In [17]:
from pydantic import BaseModel, ValidationError
import json
from openai import OpenAI

# Define your API client
client = OpenAI(api_key=api_key)

# Step 1: Define a Pydantic model
class PersonInfo(BaseModel):
    name: str
    age: int
    salary: int
    city: str

# Step 2: Get the structured output from OpenAI function call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "Extract name, age, and city from the sentence: Ritu Patil is 30 and lives in Pune. she earns $35000 per month",
        }
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "extract_person_info",
                "description": "Extract name, age, and city and salary from text",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                       # "salary": {"type": "integer"},
                        "salary": {"type": "string"},
                        "city": {"type": "string"},
                    },
                    "required": ["name", "age", "salary" "city"],
                },
            },
        }
    ],
    tool_choice="auto",
)

# Step 3: Extract and parse tool call arguments
tool_calls = response.choices[0].message.tool_calls
arguments = tool_calls[0].function.arguments
parsed = json.loads(arguments)
print(parsed)

# Step 4: Validate using Pydantic
try:
    validated = PersonInfo(**parsed)
    print("\n✅ Validated:", validated)
except ValidationError as ve:
    print("❌ Validation Error:", ve)


{'name': 'Ritu Patil', 'age': 30, 'city': 'Pune', 'salary': '$35000 per month'}
❌ Validation Error: 1 validation error for PersonInfo
salary
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='$35000 per month', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


In [25]:
print("Yearly Bonus =", parsed['salary']*2)

Yearly Bonus = $35000 per month$35000 per month


In [23]:
print(parsed['age']*2)

60
